In [84]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, shape, box, Polygon
from shapely.ops import unary_union
import os
import folium
import glob
import pyproj
from pyproj import Geod, Transformer
import laspy
import numpy as np
import subprocess
import whitebox
import pickle
import TEAK_class_declarations

# Initialize WhiteboxTools
wbt = whitebox.WhiteboxTools()

shapefile_path = '/data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer/TEAK_shapefile_no_buffer.shp'
new_shapefile_path = '/data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/'
indiv_shp_path = os.path.join(new_shapefile_path, 'indiv_shp')

new_shapefilename = 'TEAK_shapefile_no_buffer_resized.shp'
las_directory = '/data/shared/rsdata/lidar/SMfp/NEON/TEAK/TEAK_2019_smfp_discrete'

TEAK_clipped_las_path = "/data/shared/src/STV/NEON_TEAK/allen/data/clipped_las/"

# Define the CRS EPSG code for NEON TEAK site in Alaska
TEAK_CRS = 32618

objects_path = "/data/shared/src/STV/NEON_TEAK/allen/object_data"

def load_objects_from_file(filename, path=""):
    # Adjust the filename to include the subfolder
    if path:
        filename = os.path.join(path, filename)
        
    # Load the objects from file
    with open(filename, 'rb') as f:
        objects_list = pickle.load(f)
    
    return objects_list

teak_plot_objects_100 = load_objects_from_file("plot_objects_100.pkl", objects_path)
teak_plot_objects_400 = load_objects_from_file("plot_objects_400.pkl", objects_path)

In [2]:
def check_las_crs(las_dir):
    """Check the CRS of all LAS/LAZ files in the specified directory."""
    # Get all LAS and LAZ files
    las_files = glob.glob(os.path.join(las_dir, '*.las')) + glob.glob(os.path.join(las_dir, '*.laz'))
    
    crs_info = {}

    for las_file in las_files:
        with laspy.open(las_file) as las:
            # Get the header to extract CRS information
            header = las.header
            crs = header.projected_crs
            
            # Check if CRS is defined and store it
            if crs is not None:
                crs_info[os.path.basename(las_file)] = crs
            else:
                crs_info[os.path.basename(las_file)] = "No CRS defined"

    return crs_info

def get_las_bounding_box(las_file):
    """Get the bounding box of a LAS/LAZ file."""
    with laspy.open(las_file) as las:
        # Get the header to extract bounds
        header = las.header
        min_x, min_y, min_z = header.min
        max_x, max_y, max_z = header.max

        # Create a bounding box with the min and max coordinates
        bounding_box = {
            'min_x': min_x,
            'min_y': min_y,
            'max_x': max_x,
            'max_y': max_y
        }
        return bounding_box
    
def get_las_bounding_boxes(directory):
    """Get bounding boxes for all LAS/LAZ files in the specified directory."""
    las_files = glob.glob(os.path.join(directory, '*.laz'))
    laz_files = glob.glob(os.path.join(directory, '*.las'))
    all_files = las_files + laz_files
    bounding_boxes = {}

    for las_file in all_files:
        bbox = get_las_bounding_box(las_file)
        bounding_boxes[las_file] = box(bbox['min_x'], bbox['min_y'], bbox['max_x'], bbox['max_y'])
    return bounding_boxes

def check_plots_within_las(shapefile, las_dir):
    # Get LAS bounding boxes from the LAS directory
    las_bboxes = get_las_bounding_boxes(las_dir)  # Assuming this function exists and works

    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shapefile)
    gdf = remove_plots_above_threshold(gdf, "TEAK_047")
    gdf = shrink_distributed_bounding_boxes(gdf)
    save_gdf_to_shapefile(gdf, new_shapefile_path, new_shapefilename)
    split_shapefile_by_plot_id(gdf, indiv_shp_path)

    # Compute the combined bounding box of all LAS files
    combined_las_bbox = unary_union(list(las_bboxes.values()))

    fully_within = []
    not_fully_within = []

    # Iterate through each plot in the shapefile (based on plotID)
    for idx, row in gdf.iterrows():
        plot_id = row['plotID']
        plot_type = row['plotType']
        plot_dim = row['plotDim']
        shp_bbox = row['geometry']  # Bounding box of the plot geometry

        containing_las_files = []
        
        for las_file, las_bbox in las_bboxes.items():
            if las_bbox.intersects(shp_bbox):
                las_file_name = os.path.basename(las_file)
                containing_las_files.append(las_file_name)
        
        # Check if the plot's bbox is fully within the combined bounding box of the LAS files
        fully_within_check = combined_las_bbox.contains(shp_bbox)
        
        # Common data to save space
        result_data = {
            'plotID': plot_id,
            'plotType': plot_type,
            'plotDim': plot_dim,
            'las_files': containing_las_files,
            'plot_bbox': shp_bbox.bounds,  # Bounding box of the plot
            'combined_las_bbox': combined_las_bbox.bounds  # Bounding box of the combined LAS area
        }

        # Append to the appropriate list based on whether the plot is fully within the LAS bounds
        if fully_within_check:
            fully_within.append(result_data)
        else:
            not_fully_within.append(result_data)

    return fully_within, not_fully_within

def remove_plots_above_threshold(gdf, threshold):
    # Filter the GeoDataFrame to keep only the desired plotIDs
    filtered_gdf = gdf[gdf['plotID'] <= threshold].reset_index(drop=True)
    
    return filtered_gdf

def shrink_distributed_bounding_boxes(gdf, plot_type='distributed', new_size=20):
    # Iterate through each plot in the GeoDataFrame
    for idx, row in gdf.iterrows():
        if row['plotType'] == plot_type:
            # Get the bounds of the original bounding box
            min_x, min_y, max_x, max_y = row['geometry'].bounds
            
            # Calculate the center point
            center_x = (min_x + max_x) / 2
            center_y = (min_y + max_y) / 2
            
            # Create a new bounding box centered at the center point
            half_new_size = new_size / 2
            new_min_x = center_x - half_new_size
            new_max_x = center_x + half_new_size
            new_min_y = center_y - half_new_size
            new_max_y = center_y + half_new_size
            
            # Create the new bounding box
            new_bbox = box(new_min_x, new_min_y, new_max_x, new_max_y)
            
            # Update the geometry in the GeoDataFrame
            gdf.at[idx, 'geometry'] = new_bbox
            gdf.at[idx, 'plotDim'] = '20m x 20m'  # Update plot_dim to '20m x 20m'

    return gdf

def save_gdf_to_shapefile(gdf, output_directory, output_filename):
    """Save the GeoDataFrame to a shapefile in the specified directory."""
    # Create the full output path
    output_path = os.path.join(output_directory, output_filename)
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Save the GeoDataFrame to a shapefile
    gdf.to_file(output_path, driver='ESRI Shapefile')

    print(f"Shapefile saved to: {output_path}")
    
def format_results(results):
    """Format the results into a more readable structure and export to a file if needed."""
    # Convert the results dictionary into a list of dictionaries for better readability
    # Initialize a list to store formatted results
    formatted_results = []

    # Process fully within results
    for result in results:
        formatted_results.append({
            'plotID': result['plotID'],
            'Containing LAS Files': ', '.join(result['las_files']),
            'Shapefile Bounding Box (min_x, min_y, max_x, max_y)': result['plot_bbox'],
            'Combined LAS Bounding Box (min_x, min_y, max_x, max_y)': result['combined_las_bbox']
        })
    
    # Create a DataFrame for better readability
    df_results = pd.DataFrame(formatted_results)
    
    # Print the results
    print(df_results.to_string(index=False))
    
    # Optionally, export the results to a CSV file
    csv_path = os.path.join(TEAK_clipped_las_path, 'shapefile_check_results.csv')
    df_results.to_csv(csv_path, index=False)
    print(f'Results exported to {csv_path}')

def split_shapefile_by_plot_id(gdf, output_dir):
    """Split the master shapefile into individual shapefiles for each plot ID."""

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get unique plot IDs
    unique_plot_ids = gdf['plotID'].unique()
    
    for plot_id in unique_plot_ids:
        # Filter the GeoDataFrame for the current plot ID
        plot_gdf = gdf[gdf['plotID'] == plot_id]
        
        # Define the output shapefile path
        output_shapefile = os.path.join(output_dir, f'{plot_id}.shp')
        
        # Save the individual plot shapefile
        plot_gdf.to_file(output_shapefile, driver='ESRI Shapefile')
        print(f'Saved shapefile for plot ID {plot_id} to: {output_shapefile}')

def clip_las_to_geometry(las_dir, shapefile_dir, fully_within_results, output_dir):
    """Clip LAS files using the corresponding shapefiles based on fully_within results."""
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process each entry in the fully_within results
    for result in fully_within_results:
        plot_id = result['plotID']
        las_files = result['las_files']

        # Iterate through each LAS file associated with the current plot
        for las_file_name in las_files:
            # Construct the full path to the LAS file
            las_file_path = os.path.join(las_dir, las_file_name)

            # Construct the corresponding shapefile path
            shapefile_path = os.path.join(shapefile_dir, f'{plot_id}.shp')  # Using the passed shapefile directory

            # Check if the LAS file exists and the shapefile exists
            if os.path.exists(las_file_path) and os.path.exists(shapefile_path):
                # Define the output path for the clipped LAS file
                output_las_file_path = os.path.join(output_dir, f'clipped_{plot_id}_{las_file_name}')
                
                # Clip the LAS file using the shapefile
                wbt.clip_lidar_to_polygon(
                    i=las_file_path,         # The LAS file to be clipped
                    output=output_las_file_path, # The output path for the clipped LAS
                    polygons=shapefile_path             # The shapefile used for clipping
                )

                print(f'Clipped {las_file_name} using {shapefile_path}. Saved to {output_las_file_path}.')
            else:
                if not os.path.exists(las_file_path):
                    print(f'LAS file {las_file_path} not found. Skipping.')
                if not os.path.exists(shapefile_path):
                    print(f'Shapefile {shapefile_path} not found. Skipping.')


In [104]:
def print_las_header(las_file):
    """Print the header information of a LAS/LAZ file."""
    with laspy.open(las_file) as las:
        header = las.header
        print("Header Information for:", las_file)
        # Extract min and max coordinates
        min_x, min_y, min_z = header.min
        max_x, max_y, max_z = header.max
        
        
        
        # Print all attributes of the header
        for attribute, value in header.__dict__.items():
            print(f"{attribute}: {value}")
        
        print("")
        
        print(header)
        
def print_headers_in_directory(las_dir):
    """Print headers for all LAS/LAZ files in the specified directory."""
    # Get all LAS and LAZ files
    las_files = glob.glob(os.path.join(las_dir, '*.las')) + glob.glob(os.path.join(las_dir, '*.laz'))
    
    for las_file in las_files:
        print_las_header(las_file)

print_headers_in_directory(las_directory)

Header Information for: /data/shared/rsdata/lidar/SMfp/NEON/TEAK/TEAK_2019_smfp_discrete/NEON_D17_TEAK_DP1_312000_4091000_classified_point_cloud_colorized.las
file_source_id: 211
global_encoding: <laspy.header.GlobalEncoding object at 0x7fe8b9147830>
uuid: 00000000-0000-0000-0000-000000000000
_version: 1.3
system_identifier: LAStools (c) by rapidlasso GmbH
generating_software: lascolor (190812) commercial
_point_format: <PointFormat(3, 0 bytes of extra dims)>
creation_date: 2019-07-25
point_count: 74853
scales: [0.01 0.01 0.01]
offsets: [ 300000. 4000000.       0.]
maxs: [3.12999990e+05 4.09199998e+06 1.67942000e+03]
mins: [3.12799130e+05 4.09140332e+06 9.93260000e+02]
number_of_points_by_return: [57929 15283  1586    55     0     0     0     0     0     0     0     0
     0     0     0]
_vlrs: [<GeoKeyDirectoryVlr(4 geo_keys)>, <VLR(user_id: 'LAStools', record_id: '10', data len: 28)>]
extra_header_bytes: b''
extra_vlr_bytes: b''
start_of_waveform_data_packet_record: 0
start_of_first_

In [87]:
fully_within, not_fully_within = check_plots_within_las(shapefile_path, las_directory)

format_results(fully_within)

Shapefile saved to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/TEAK_shapefile_no_buffer_resized.shp
Saved shapefile for plot ID TEAK_001 to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/indiv_shp/TEAK_001.shp
Saved shapefile for plot ID TEAK_002 to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/indiv_shp/TEAK_002.shp
Saved shapefile for plot ID TEAK_003 to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/indiv_shp/TEAK_003.shp
Saved shapefile for plot ID TEAK_005 to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/indiv_shp/TEAK_005.shp
Saved shapefile for plot ID TEAK_006 to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/indiv_shp/TEAK_006.shp
Saved shapefile for plot ID TEAK_007 to: /data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefil

In [92]:
clip_las_to_geometry(las_directory, indiv_shp_path, fully_within, TEAK_clipped_las_path)

./whitebox_tools --run="ClipLidarToPolygon" --input='/data/shared/rsdata/lidar/SMfp/NEON/TEAK/TEAK_2019_smfp_discrete/NEON_D17_TEAK_DP1_320000_4094000_classified_point_cloud_colorized.las' --polygons='/data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/indiv_shp/TEAK_001.shp' --output='/data/shared/src/STV/NEON_TEAK/allen/data/clipped_las/clipped_TEAK_001_NEON_D17_TEAK_DP1_320000_4094000_classified_point_cloud_colorized.las' -v --compress_rasters=False

*********************************
* Welcome to ClipLidarToPolygon *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************


Reading data...
Performing clip...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Progress: 48%
Progress: 49%
Progress: 50%
Progress: 51%
Progress: 52%
Progress: 53%
Progress: 54%
Progress: 55%
Progress: 56%
Progress: 57%
Progress: 58%
Progress: 59%
Progress: 60%
Progress: 61%
Progress: 62%
Progress: 63%
Progress: 64%
Progress: 65%
Progress: 66%
Progress: 67%
Progress: 68%
Progress:

In [ ]:
def utm_to_latlon(easting, northing, crs):
    """Convert UTM coordinates to latitude and longitude using the CRS from the shapefile."""
    # Create CRS object
    utm_crs = pyproj.CRS(crs)
    latlon_crs = pyproj.CRS(proj='latlong', ellps='WGS84')
    
    # Create a transformer for converting coordinates
    transformer = Transformer.from_crs(utm_crs, latlon_crs, always_xy=True)
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

def calculate_map_center(fully_within_results, crs):
    """Calculate the center coordinates from fully_within results and convert to lat/lon."""
    total_x = 0
    total_y = 0
    count = 0

    for result in fully_within_results:
        # Extract the plot bounding box
        plot_bbox = result['plot_bbox']  # This should be a bounding box [min_x, min_y, max_x, max_y]
        
        # Calculate the center of the bounding box in UTM
        center_x = (plot_bbox[0] + plot_bbox[2]) / 2  # (min_x + max_x) / 2
        center_y = (plot_bbox[1] + plot_bbox[3]) / 2  # (min_y + max_y) / 2

        total_x += center_x
        total_y += center_y
        count += 1

    # Calculate average center coordinates in UTM
    if count > 0:
        avg_center_x = total_x / count
        avg_center_y = total_y / count
        
        # Convert the average UTM center coordinates to lat/lon
        lat, lon = utm_to_latlon(avg_center_x, avg_center_y, crs)
        
        return lat, lon
    return None, None

def plot_boundaries_from_lasfiles(map_object, las_file_dir, crs_epsg):
    """Plot the bounding boxes of LAS/LAZ files on a Folium map."""
    # Retrieve bounding boxes from LAS files
    las_bboxes = get_las_bounding_boxes(las_file_dir)
    
    # Initialize the transformer for the given CRS
    transformer = Transformer.from_crs(crs_epsg, "EPSG:4326", always_xy=True)

    # Plot each bounding box
    for las_file, bbox in las_bboxes.items():
        # Convert bounding box coordinates to lat/lon
        min_x, min_y = bbox.bounds[0], bbox.bounds[1]
        max_x, max_y = bbox.bounds[2], bbox.bounds[3]
        
        min_lon, min_lat  = transformer.transform(min_x, min_y)
        max_lon, max_lat  = transformer.transform(max_x, max_y)

        # Calculate the center of the bounding box
        center_lon = (min_lon + max_lon) / 2
        center_lat = (min_lat + max_lat) / 2
    
        # Extract just the file name (not the full path)
        file_name = os.path.basename(las_file)

        # Add a rectangle to the map
        folium.Polygon(
            locations=[
                [min_lat, min_lon],
                [min_lat, max_lon],
                [max_lat, max_lon],
                [max_lat, min_lon],
                [min_lat, min_lon]
            ],
            color='blue',
            fill=True,
            fill_opacity=0.3,
            # popup=f"LAS File: {las_file}"
        ).add_to(map_object)

        # Add a marker at the center of the bounding box
        folium.Marker(
            location=[center_lat, center_lon],
            popup=file_name,
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_object)
        
        # Debugging statements
        print(f"LAS File: {las_file}")
        print(f"Bounding Box (EPSG:{crs_epsg}): {bbox}")
        print(f"Converted to Lat/Lon: ({min_lat}, {min_lon}) to ({max_lat}, {max_lon})")

# Calculate the center of the map from the fully_within results
map_center_lat, map_center_lon = calculate_map_center(fully_within, TEAK_CRS)

map_object = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=10)
plot_boundaries_from_lasfiles(map_object, las_directory, TEAK_CRS)